In [74]:
# Torch
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import torch.utils.data as data

# image handling
import cv2

# metadata handling
import numpy as np
import pandas as pd
import os.path as osp
import ast
from sklearn import model_selection
from itertools import product as product
from math import sqrt as sqrt

rootpath = '../'
df = pd.read_csv(osp.join(rootpath, "train.csv"))
# df = df[df.annotations.apply(len) > 0].reset_index(drop=True)
# df_tr, df_va = model_selection.train_test_split(df, test_size=0.25, random_state=42, shuffle=True)

In [75]:
# アノテーション数
df['annotations'] = df['annotations'].apply(ast.literal_eval)
df['n_annotations'] = df['annotations'].str.len()
df['has_annotations'] = df['annotations'].str.len() > 0
df['doesnt_have_annotations'] = df['annotations'].str.len() == 0

# 物体の有無によるシーケンスの分割
df['start_cut_here'] = df['has_annotations'] & df['doesnt_have_annotations'].shift(1)  & df['doesnt_have_annotations'].shift(2)
df['end_cut_here'] = df['doesnt_have_annotations'] & df['has_annotations'].shift(1)  & df['has_annotations'].shift(2)
df['sequence_change'] = df['sequence'] != df['sequence'].shift(1)
df['last_row'] =  df.index == len(df)-1
df['cut_here'] = df['start_cut_here'] | df['end_cut_here'] | df['sequence_change'] | df['last_row']
start_idx = 0
for subsequence_id, end_idx in enumerate(df[df['cut_here']].index):
    df.loc[start_idx:end_idx, 'subsequence_id'] = subsequence_id
    start_idx = end_idx
df['subsequence_id'] = df['subsequence_id'].astype(int)
drop_cols = ['start_cut_here', 'end_cut_here', 'sequence_change', 'last_row', 'cut_here', 'doesnt_have_annotations']
df = df.drop(drop_cols, axis=1)

# 物体の有無による階層化を用いたsubsequenceのfold振り分け
df_split  = df.groupby("subsequence_id").agg({'has_annotations': 'max', 'video_frame': 'count'}).astype(int).reset_index()
n_splits = 5
kf = model_selection.StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
for fold_id, (_, val_idx) in enumerate(kf.split(df_split['subsequence_id'], y=df_split["has_annotations"])):
    subseq_val_idx = df_split['subsequence_id'].iloc[val_idx]
    df.loc[df['subsequence_id'].isin(subseq_val_idx), 'fold'] = fold_id
    print(f"fold {fold_id} : {subseq_val_idx.values}")
    
df['fold'] = df['fold'].astype(int)
print('\n# of images / fold')
display(df['fold'].value_counts(dropna=False))

fold 0 : [  2  18  30  38  44  50  53  57  58  61  64  65  67  75  84  87  88  91
 100 102 111 118 121 122 123 125 128 134]
fold 1 : [  5   9  10  11  24  28  31  35  39  49  60  77  79  85  89  90  92  93
  95  96 101 103 104 108 110 113 126 127]
fold 2 : [  6   7  16  19  20  22  23  26  27  41  45  52  59  66  72  73  76  78
  82  83  98 106 107 114 116 124 137]
fold 3 : [  1   3   4   8  13  14  21  34  36  37  40  43  46  51  55  56  70  74
  80  81  86 105 112 117 120 129 135]
fold 4 : [ 12  15  17  25  29  32  33  42  47  48  54  62  63  68  69  71  94  97
  99 109 115 119 130 131 132 133 136]

# of images / fold


3    7742
1    5415
2    3868
4    3332
0    3144
Name: fold, dtype: int64

In [76]:
df

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,n_annotations,has_annotations,subsequence_id,fold
0,0,40258,0,0,0-0,[],0,False,1,3
1,0,40258,1,1,0-1,[],0,False,1,3
2,0,40258,2,2,0-2,[],0,False,1,3
3,0,40258,3,3,0-3,[],0,False,1,3
4,0,40258,4,4,0-4,[],0,False,1,3
...,...,...,...,...,...,...,...,...,...,...
23496,2,29859,10755,2983,2-10755,[],0,False,137,2
23497,2,29859,10756,2984,2-10756,[],0,False,137,2
23498,2,29859,10757,2985,2-10757,[],0,False,137,2
23499,2,29859,10758,2986,2-10758,[],0,False,137,2


In [84]:
fold = 0
train_id_names = df[df.fold != fold].reset_index(drop=True)["image_id"]
val_id_names = df[df.fold == fold].reset_index(drop=True)["image_id"]

# 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
imgpath_template = osp.join(rootpath, "train_image", "%s.jpg")
annopath_template = osp.join(rootpath, "labels", "%s.txt")

# 訓練データの画像ファイルとアノテーションファイルへのパスリストを作成
train_img_list = list()
train_anno_list = list()

for fname in train_id_names:
    img_path = imgpath_template % fname
    anno_path = annopath_template % fname
    train_img_list.append(img_path)
    train_anno_list.append(anno_path)

In [88]:
# 検証データの画像ファイルとアノテーションファイルへのパスリストを作成
val_img_list = list()
val_anno_list = list()

for fname in val_id_names:
    img_path = imgpath_template % fname  # 画像のパス
    anno_path = annopath_template % fname  # アノテーションのパス
    val_img_list.append(img_path)  # リストに追加
    val_anno_list.append(anno_path)  # リストに追加

In [160]:
bboxes = list()
try:
    with open("../labels\\0-35.txt") as f:
        # 画像内にある物体（object）の数だけループする
        for obj in f.readlines():
            bbox = obj.split()
            class_, x_min, x_max, width, height = int(bbox[0]), float(bbox[1]), float(bbox[2]), float(bbox[3]), float(bbox[4])
            bboxes += [[x_min, x_max, width, height, class_]]
    bboxes = np.array(bboxes)
except:
    bboxes = np.empty((0,0))
print(bboxes)

[[0.436719 0.252778 0.060937 0.086111 0.      ]
 [0.489844 0.305556 0.045312 0.044444 0.      ]]


In [161]:
bboxes

array([[0.436719, 0.252778, 0.060937, 0.086111, 0.      ],
       [0.489844, 0.305556, 0.045312, 0.044444, 0.      ]])